In [1]:
require 'hdf5'

{
  H5Z_FILTER_CONFIG_ENCODE_ENABLED : 1
  H5F_ACC_RDWR : 1
  _getTorchType : function: 0x0e27b0c0
  H5F_OBJ_FILE : 1
  H5S_ALL : 0
  H5F_OBJ_GROUP : 4
  C : userdata: 0x0e660ee0
  H5P_DEFAULT : 0
  _describeObject : function: 0x0e65fc48
  H5Z_FILTER_NBIT : 5
  _debugMode : false
  _getObjectType : function: 0x0e6604b0
  H5F_OBJ_ALL : 31
  _getObjectName : function: 0x0e305480
  version : 
    {
      1 : 1
  

    2 : 8
      3 : 15
    }
  H5Z_FILTER_SHUFFLE : 2
  HDF5Group : table: 0x0e288838
  open : function: 0x0e2e2738
  H5Z_FILTER_SZIP : 4
  H5F_OBJ_ATTR : 16
  H5Z_FILTER_FLETCHER32 : 3
  H5F_OBJ_DATATYPE : 8
  debugMode : function: 0x0e2f8610
  H5F_ACC_EXCL : 4
  H5Z_FILTER_NONE : 0
  _testUtils : 
    {
      deepAlmostEq : function: 0x0e2f85d0
      withTmpDir : function: 0x0e2f8530
    }
  _nativeTypeForTensorType : function: 0x0e25b320
  H5F_ACC_TRUNC : 2
  _config : 
    {
      HDF5_INCLUDE_PATH : /Users/nicolasdrizard/anaconda/include
      HDF5_LIBRARIES : /Users/nicolasdrizard/anaconda/lib/libhdf5.dylib;/Users/nicolasdrizard/anaconda/lib/libhdf5_hl.dylib;/Users/nicolasdrizard/anaconda/lib/libhdf5.dylib;/Users/nicolasdrizard/anaconda/lib/libz.dylib;/usr/lib/libdl.dylib;/usr/lib/libm.dylib
    }
  _loadObject : function: 0x0e2e2718
  DataSetOptions : table: 0x0e2b5940
  HDF5DataSet : table: 0x0e293cf0
  H5Z_FILTER_RESERVED : 256
  _logger : 
    {
      error : function: 0x0e2f

  NATIVE_INT_FAST64 : 50331686
      NATIVE_UINT_FAST64 : 50331687
      NATIVE_UINT_LEAST64 : 50331685
      NATIVE_INT_LEAST64 : 50331684
      ENUM : 8
      NATIVE_UINT64 : 50331683
      NATIVE_HBOOL : 50331701
      NATIVE_ULONG : 50331663
      NATIVE_INT_FAST32 : 50331680
      NATIVE_HADDR : 50331697
      NATIVE_UINT : 50331661
      NCLASSES : 11
      NATIVE_UINT_LEAST32 : 50331679
      NATIVE_INT_LEAST32 : 50331678
      STD_U64LE : 50331722
      NATIVE_UINT32 : 50331677
      NATIVE_SHORT : 50331658
      NATIVE_INT32 : 50331676
      VLEN : 9
      ARRAY : 10
      STD_U16LE : 50331718
      STD_B16LE : 50331726
      STD_I64LE : 50331714
      NATIVE_UINT16 : 50331671
      NATIVE_UINT_FAST8 : 50331669
      STD_B16BE : 50331727
      NATIVE_INT_FAST8 : 50331668
      FLOAT : 1
      REFERENCE : 7
      STD_U32LE : 50331720
      NATIVE_USHORT : 50331659
      NATIVE_ULLONG : 50331689
      NATIVE_INT8 : 50331664
      IEEE_F32LE : 50331702
      STD_U8BE : 50331717
 

    STD_B64BE : 50331731
      STD_U8LE : 50331716
      STD_I16BE : 50331711
      STD_B8BE : 50331725
      STRING : 3
      STD_I16LE : 50331710
      STD_U64BE : 50331723
    }
  HDF5File : table: 0x0e2cfbc0
  H5Z_FILTER_SCALEOFFSET : 6
  H5Z_FILTER_DEFLATE : 1
  H5F_ACC_CREAT : 16
  H5F_UNLIMITED : 18446744073709551615ULL
  _deflateAvailable : function: 0x0e2c6ff8
  _inDebugMode : function: 0x0e2f8650
  H5F_OBJ_LOCAL : 32
  H5S_SELECT_SET : 0
  _datatypeName : function: 0x0e2c70d0
  H5F_ACC_DEBUG : 8
  H5F_OBJ_DATASET : 2
}


# 1) Probability Computation with Greedy 

In [29]:
function get_F_count(F, N)
    -- Build the mapping from (N-1)gram to row index
    -- and the count matrix F_count: (num_context, 2)
    local ngram_to_ind = {}
    local key
    for i=1,F:size(1) do
        key = tostring(F[{i,1}])
        -- Building key
        for k = 2,N-1 do
            key = key .. '-' .. tostring(F[{i,k}])
        end
        ngram_to_ind[key] = i
    end
    return F:narrow(2,N,2), ngram_to_ind
end

In [6]:
function compute_perplexity(gram_input, F_count, ngram_to_ind, N)
    -- Compute perplexity on entry with space
    local perp = 0
    local context = torch.zeros(N-1)
    local probability = torch.zeros(2)
    -- Do not predict for the last char
    --for i=1,gram_input:size(1)-N do
    local size=gram_input:size(1) - (N-1)
    for i=1,size do
        context:copy(gram_input:narrow(1,i,N-1))
        -- Line where the model appears
        probability:copy(compute_count_based_probability(context, F_count, ngram_to_ind, 1))
        if gram_input[i+(N-1)] == 1 then
            right_proba = probability[1]
            --print('space')
            --print(right_proba)
        else
            right_proba = probability[2]
        end
        perp = perp + math.log(right_proba)
    end
    perp = math.exp(-perp/size)
    --perp = math.exp(-perp/(gram_input:size(1)-N))
    return perp
end

In [130]:
print('Train')
print(compute_perplexity(input_data_train, F_count, ngram_to_ind, N))

Train	


1.3967959853446	


In [4]:
-- Compute proba distribution over (space, char) for the context
-- F is here the count matrix (num_context, 2)
function compute_count_based_probability(context, F_count, ngram_to_ind, alpha)
    local probability = torch.zeros(2)
    -- Building key, ie (N-1)gram (from i to i+(N-2))
    local key = tostring(context[1])
    for k = 2,context:size(1) do
        key = key .. '-' .. tostring(context[k])
    end
    -- If (N-1)gram never seen, prior distribution
    if (ngram_to_ind[key] ~= nil) then
        -- index of the current (n-1)gram in the F matrix
        local index = ngram_to_ind[key]
        probability:copy(F_count:narrow(1,index,1))
        -- Adding smoothing
        probability:add(alpha)
    -- Case unseen context
    else
        -- Prior
        probability:copy(torch.DoubleTensor({F_count:narrow(2,1,1):sum(), F_count:narrow(2,2,1):sum()}))
    end
    return probability:div(probability:sum())
end

In [5]:
function predict_count_based_greedy(gram_input, F_count, ngram_to_ind, N)
    -- Next Position to fill in predictions
    local position = N
    -- We allocate the maximum of memory that could be needed
    -- Default value is -1 (to know where predictions end afterwards)
    local predictions = torch.ones(2*(gram_input:size(1) - N)):mul(-1)
    -- Copy the first (N-1) gram
    predictions:narrow(1,1,N-1):copy(gram_input:narrow(1,1,N-1))
    local probability = torch.zeros(2)
    local context = torch.zeros(N-1)

    -- Build mapping
    for i=1,gram_input:size(1)-N do
        -- Compute proba for next char
        context:copy(predictions:narrow(1,position-(N-1),N-1))
        -- Line where the model appears
        probability:copy(compute_count_based_probability(context, F_count, ngram_to_ind, 1))
        m,a = probability:max(1)

        -- Case space predicted
        if (a[1] == 1) then
            predictions[position] = 1
            position = position +1
        end

        -- Copying next character
        predictions[position] = gram_input[i+N-1]
        position = position +1
    end
    -- Adding last character (</s>)
    predictions[position] = gram_input[gram_input:size(1)]
    -- Cutting the output
    return predictions:narrow(1,1,position)
end   

In [19]:
myFile = hdf5.open('../data_preprocessed/5-grams.hdf5','r')
data = myFile:all()
F_train = data['F_train']
input_data_valid = data['input_data_valid']
input_data_train = data['input_data_train']
input_data_valid_nospace = data['input_data_valid_nospace']
input_data_test = data['input_data_test']
myFile:close()

In [20]:
F_count, ngram_to_ind = get_F_count(F_train, N)

In [21]:
print(F_count:narrow(2,1,1):sum())
print(F_count:narrow(2,2,1):sum())

100961	
498942	


In [15]:
N = 5
gram_input_no_space = input_data_valid_nospace
predictions_new = predict_count_based_greedy(gram_input_no_space, F_count, ngram_to_ind, N)

In [86]:
for N=2,6 do
    myFile = hdf5.open('../data_preprocessed/'..tostring(N)..'-grams.hdf5','r')
    data = myFile:all()
    F_train = data['F_train']
    input_data_valid = data['input_data_valid']
    input_data_train = data['input_data_train']
    input_data_valid_nospace = data['input_data_valid_nospace']
    myFile:close()

    F_count, ngram_to_ind = get_F_count(F_train, N)
    print(N)
    print('Train')
    print(compute_perplexity(input_data_train, F_count, ngram_to_ind, N))
    print('Valid')
    print(compute_perplexity(input_data_valid, F_count, ngram_to_ind, N))
end

2	
Train	


1.3967959853446	
Valid	


1.4000666169436	


3	
Train	


1.2680508928739	
Valid	


...

1.1467907206376	
Valid	


1.1643258177066	


6	
Train	


1.1618916590178	
Valid	


1.1911975320322	


# 2) Kaggle Submission

In [191]:
-- Kaggle Prediction
N = 5

myFile = hdf5.open('../data_preprocessed/'..tostring(N)..'-grams.hdf5','r')
data = myFile:all()
F_train = data['F_train']
input_data_test = data['input_data_test']
input_data_valid = data['input_data_valid']
input_data_train = data['input_data_train']
input_data_valid_nospace = data['input_data_valid_nospace']
myFile:close()

F_count, ngram_to_ind = get_F_count(F_train, N)
predictions_test = predict_count_based_greedy(input_data_test, F_count, ngram_to_ind, N)

In [192]:
input_data_test:narrow(1,input_data_test:size(1),1)

 2
[torch.LongTensor of size 1]



In [199]:
function get_kaggle_format(predictions_test, N)
    -- Counting sentences
    local num_sentence = 0
    for i=N-1,predictions_test:size(1) do
        if predictions_test[i] == 2 then
            num_sentence = num_sentence + 1
        end
    end

    -- Counting space per sentence
    local num_spaces = torch.DoubleTensor(num_sentence,2)
    local row = 1
    local count_space = 0
    for i=N-1,predictions_test:size(1) do
        if predictions_test[i] == 2 then
            num_spaces[{row, 1}] = row
            num_spaces[{row, 2}] = count_space
            count_space = 0
            row = row + 1
        elseif predictions_test[i] == 1 then
            count_space = count_space + 1
        end
    end
    return num_spaces
end

In [200]:
num_spaces = get_kaggle_format(predictions_test, N)

In [2]:
-- Saving the Kaggle format output
myFile = hdf5.open('../submission/pred_test_greedi_count_5', 'w')
myFile:write('num_spaces', num_spaces)
myFile:close()

# 3) Viterbi coding

In [82]:
-- In pi matrix col1: space; col2: char

function predict_count_based_viterbi(gram_input, F_count, ngram_to_ind, N)
    -- Backpointer
    local score
    local bp = torch.zeros(gram_input:size(1) + 1, 2)
    local context = torch.DoubleTensor(1)
    local y_hat = torch.DoubleTensor(2)
    local pi = torch.ones(gram_input:size(1) + 1, 2):mul(-9999)
    -- Initialization
    pi[{1,1}] = 0
    -- i is shifted
    for i=2,gram_input:size(1)+1 do
        for c_prev =1,2 do
            -- Precompute y_hat(c_prev)
            if c_prev == 1 then
                context[1] = c_prev
            else
                context[1] = gram_input[i-1]
            end
            -- Line where the model appears
            y_hat:copy(compute_probability(context, F_count, ngram_to_ind, 1))

            for c_current =1,2 do
                score = pi[{i-1, c_prev}] + math.log(y_hat[c_current])
                if score > pi[{i, c_current}] then
                    pi[{i, c_current}] = score
                    bp[{i, c_current}] = c_prev
                end
            end
        end
    end
    return pi, bp
end

In [70]:
-- Building the sequences from the backpointer
function build_sequences_from_bp(bp, gram_input)
    local predictions = torch.DoubleTensor(2*gram_input:size(1))
    -- Next position to fill in predictions (have to do it backward)
    local position = 2*gram_input:size(1)
    local col = 2
    -- Loop until the 3rd position (because 2nd is the first one, could be set by hand)
    for i=bp:size(1),3,-1 do
        -- coming from a space
        if bp[i][col] == 1 then
            predictions[position] = 1
            position = position - 1
            col = 1
        else
            col = 2
        end
        -- index i is shifted of 1 wrt local index in gram_input
        predictions[position] = gram_input[i-1]
        position = position - 1
    end
    -- Beginnning of gram_input set
    predictions[position] = gram_input[1]
    position = position - 1

    return predictions:narrow(1,position+1,predictions:size(1)-position)
end

In [83]:
N = 2

myFile = hdf5.open('../data_preprocessed/'..tostring(N)..'-grams.hdf5','r')
data = myFile:all()
F_train = data['F_train']
input_data_test = data['input_data_test']
input_data_valid = data['input_data_valid']
input_data_train = data['input_data_train']
input_data_valid_nospace = data['input_data_valid_nospace']
myFile:close()

-- Dynamic Programming version for bigram
F_count, ngram_to_ind = get_F_count(F_train, N)
gram_input = input_data_test

pi, bp = predict_count_based_viterbi(gram_input, F_count, ngram_to_ind, N)
pred = build_sequences_from_bp(bp, gram_input)

In [84]:
num_spaces_viterbi = get_kaggle_format(pred, N)

In [85]:
print('Number of spaces with Viterbi in test')
print(num_spaces_viterbi:narrow(2,2,1):sum())
print('Number of spaces with Greedy in test')
print(num_spaces:narrow(2,2,1):sum())

Number of spaces with Viterbi in test	
3218	
Number of spaces with Greedy in test	
64456	


# 4) Viterbi on 3-grams

In [126]:
-- In pi matrix col1: space; col2: char

function predict_count_based_viterbi_trigram(gram_input, F_count, ngram_to_ind, N)
    -- Backpointer
    local score
    local bp = torch.zeros(gram_input:size(1) + 1, 3)
    local context = torch.DoubleTensor(2)
    local y_hat = torch.DoubleTensor(2)
    -- pi is built as ('char-space', 'char-char', 'space-char')
    -- corresponding index in the context
    local pi = torch.ones(gram_input:size(1) + 1, 3):mul(-999999999)
    -- Initialization
    pi[{2,1}] = 0
    --pi[{2,2}] = 0
    --pi[{2,3}] = 0
    -- We need to start at the first trigram
    for i=3,gram_input:size(1)+1 do
        for c_prev =1,3 do
            -- Precompute y_hat(c_prev)
            if c_prev == 1 then
                context[1] = gram_input[i-2]
                context[2] = 1
            elseif c_prev == 2 then
                context[1] = gram_input[i-2]
                context[2] = gram_input[i-1]
            else
                context[1] = 1
                context[2] = gram_input[i-1]
            end
            -- Line where the model appears
            y_hat:copy(compute_probability(context, F_count, ngram_to_ind, 1))

            -- cannot have 2 spaces in a row: from 1 goes to 3 necessarily
            if c_prev == 1 then
                pi[{i, 3}] = pi[{i-1, c_prev}] + math.log(y_hat[2])
                bp[{i, 3}] = c_prev
            else
                -- last char is necessarily 'char' so
                -- 1: space predicted (ie 'char-space')
                -- 2: char predicted (ie 'char-char')
                for c_current =1,2 do
                    score = pi[{i-1, c_prev}] + math.log(y_hat[c_current])
                    if score > pi[{i, c_current}] then
                        pi[{i, c_current}] = score
                        bp[{i, c_current}] = c_prev
                    end
                end
            end
        end
    end
    return pi, bp
end

In [136]:
-- Building the sequences from the backpointer
-- We start the sequence by the ('char'-'char') configuration
-- as we know it's the only one possible
function build_sequences_from_bp_trigram(bp, gram_input)
    local predictions = torch.DoubleTensor(2*gram_input:size(1))
    -- Next position to fill in predictions (have to do it backward)
    local position = 2*gram_input:size(1)
    local col = 2
    -- Loop until the 4th position 
    for i=bp:size(1),4,-1 do
        -- coming from a space
        if bp[i][col] == 1 then
            predictions[position] = 1
            position = position - 1
        end
        col = bp[i][col]
        -- index i is shifted of 1 wrt local index in gram_input
        predictions[position] = gram_input[i-1]
        position = position - 1
    end
    -- Beginnning of gram_input set
    predictions[position] = gram_input[2]
    position = position - 1
    predictions[position] = gram_input[1]
    position = position - 1

    return predictions:narrow(1,position+1,predictions:size(1)-position)
end

In [143]:
N = 3

myFile = hdf5.open('../data_preprocessed/'..tostring(N)..'-grams.hdf5','r')
data = myFile:all()
F_train = data['F_train']
input_data_test = data['input_data_test']
input_data_valid = data['input_data_valid']
input_data_train = data['input_data_train']
input_data_valid_nospace = data['input_data_valid_nospace']
myFile:close()

-- Dynamic Programming version for bigram
F_count, ngram_to_ind = get_F_count(F_train, N)
gram_input = input_data_test

In [144]:
pi_tri, bp_tri = predict_count_based_viterbi_trigram(gram_input, F_count, ngram_to_ind, N)
---pred = build_sequences_from_bp(bp, gram_input)

In [145]:
seq_tri = build_sequences_from_bp_trigram(bp_tri, gram_input)

In [148]:
num_spaces_tri = get_kaggle_format(seq_tri, N)
print('Number of spaces with Viterbi trigram in test')
print(num_spaces_tri:narrow(2,2,1):sum())
-- The number is closer to the Greedy case!!

Number of spaces with Viterbi trigram in test	
51832	


In [149]:
-- Saving the Kaggle format output
myFile = hdf5.open('../submission/pred_test_viterbi_trigram', 'w')
myFile:write('num_spaces', num_spaces_tri)
myFile:close()